# Demo.ipynb

Written by Taiob Ali  
SqlWorldWide.com

This script will create:

- A resource group
- One logical Azure SQL server
- Multiple empty Azure SQL Databases
- One Storage Account
- One Storage Container
- Azure Migrate Service
- Azure Database Migration Service

Reference I took help from:

- [Azure PowerShell samples for Azure SQL Database and Azure SQL Managed Instance](https://docs.microsoft.com/en-us/azure/azure-sql/database/powershell-script-content-guide?tabs=single-database)
- [PowerShell One Liner: Get External/Public IP Address](https://gallery.technet.microsoft.com/scriptcenter/Get-ExternalPublic-IP-c1b601bb)
- [Azure SQL Database and Azure Synapse IP firewall rules](https://docs.microsoft.com/en-us/azure/azure-sql/database/firewall-configure#manage-firewall-rules-using-azure-cli)
- [Use PowerShell to create a managed instance](https://docs.microsoft.com/en-us/azure/azure-sql/managed-instance/scripts/create-configure-managed-instance-powershell)

Pre-Requsite:

- Install Data Migration Assistant
- Install Visual Studio
- Install Azure Data Studio
    - Azure SQL Migration Extension installed and enabled
- Azure storage explorer
- sqlpackage installed
- Reuired AZ Module installed
- Create a backup of Source database with checksum to the above storage container
- Create a Azure SQL virtual machine
    - Make approprite inbound port rules

PowerShell 7.0.6 LTS and PowerShell 7.1.3 or higher is the recommended version of PowerShell for use with the Az PowerShell module on all platforms.  
Check your PowerShell version and update as required

In [ ]:
$PSVersionTable.PSVersion

If you've disabled module autoloading, manually import the module with `Import-Module -Name Az`. Because of the way the module is structured, this can take a few seconds.

In [ ]:
Import-Module -Name Az

Sign in to Azure

In [ ]:
Connect-AzAccount

If you need to see the list of your subscription

```
$SubscriptionList=Get-AzSubscription$SubscriptionList
```

Use below code if you have multiple subscription and you want to use a particular one

In [ ]:
Set-AzContext -SubscriptionId '18d92f52-ac34-4379-ab8b-5a5106f1c54e'

Declare Variables

In [ ]:
$resourceGroupName = "cassugresourcegroup"
$location = "East US" 
$sqlServerName = "cassugsqlserver"
$databaseDma = "databaseDma"
$databaseMigrationService = "databaseMigrationService"
$databaseVS = "databaseVisualStudio"

$azureMigrateName = "cassugAzureMigrate"
$vNetName = "migrationServiceVnet"
$databaseMigrationService = "cassugDatabaseMigrationService"

$storageAccountName = "cassugstorageaccount"
$storageContainer = "cassugbacpac"
$StorageKeyType = "StorageAccessKey"
$databasebacpac_1 = "cassugDatabasebacpac1"
$databasebacpac_2 = "cassugDatabasebacpac2"

$ipinfo = Invoke-RestMethod http://ipinfo.io/json 
$startip = $ipinfo.ip
$endip = $ipinfo.ip 
$adminlogin = "taiob"
$password = Get-Content "C:\password.txt"

Check if Resource group exists. If exist delete the resource group.

Create a new Resource group

In [ ]:
$resGrpChk = Get-AzResourceGroup `
    -Name $resourceGroupName `
    -ErrorVariable notPresent `
    -ErrorAction SilentlyContinue

if ($resGrpChk) {  
    #Delete resource group
    Remove-AzResourceGroup `
        -Name $resourceGroupName 
    Write-Host 'Resource group deleted' `
        -fore white `
        -back green
}
else {
    New-AzResourceGroup `
    -Name $resourceGroupName `
    -Location $location   
}

Create a Azure SQL Server at Resource Group rgeuazbootcamp (East US)

In [ ]:
New-AzSqlServer `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -Location $location `
  -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
  -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force)) 

Configure Firewall rule for the above server

In [ ]:
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -FirewallRuleName "TaiobDemoMachine" `
  -StartIpAddress $startip `
  -EndIpAddress $endip

# This is done to allow access to Azure Services
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -FirewallRuleName "AllowAzureService" `
  -StartIpAddress 0.0.0.0 `
  -EndIpAddress 0.0.0.0

Creating an Azure SQL Database to use with Data Migration Assistant

Puchase Model: vCpu  
Service Tier : General Purpose

In [ ]:
New-Azsqldatabase  `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -DatabaseName $databasedma `
  -RequestedServiceObjectiveName "GP_Gen5_2" `
  -MaxSizeBytes 5368709120

Creating an Azure SQL Database to use with Azure Database Migration Service

Puchase Model: vCpu  
Service Tier : General Purpose

In [ ]:
New-Azsqldatabase  `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -DatabaseName $databaseMigrationService `
  -RequestedServiceObjectiveName "GP_Gen5_2" `
  -MaxSizeBytes 5368709120

Creating an Azure SQL Database to use with Visual Studio Project

Puchase Model: vCpu  
Service Tier : General Purpose

In [ ]:
New-Azsqldatabase  `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -DatabaseName $databaseVS `
  -RequestedServiceObjectiveName "GP_Gen5_2" `
  -MaxSizeBytes 5368709120

Create storage account

In [ ]:
$storageaccount = New-AzStorageAccount `
-ResourceGroupName $resourceGroupName `
-AccountName $storageAccountName `
-Location $location `
-SkuName Standard_GRS

Create a Azure Storage Container

In [ ]:
New-AzStorageContainer -Name $storageContainer -Context $storageaccount.Context -Permission Container

Create Azure Migrate Project

Note: For location I am not using the variable. This service is not available in East US

In [ ]:
New-AzMigrateProject  `
-Location "Central US" `
-Name $azureMigrateName  `
-ResourceGroupName $resourceGroupName

Create an Azure Database Migration Service

In [ ]:
$vNetAddressPrefix = "10.0.0.0/16"
$defaultSubnetAddressPrefix = "10.0.0.0/24"
$miSubnetName = "migrationServiceSubnet-$(Get-Random)"
$miSubnetAddressPrefix = "10.0.0.0/24"

$networkSecurityGroupMiManagementService = New-AzNetworkSecurityGroup `
  -Name 'myNetworkSecurityGroupAzureDataMigrationService' `
  -ResourceGroupName $resourceGroupName `
  -location $location

$routeTableMiManagementService = New-AzRouteTable `
  -Name 'myRouteTableAzureDataMigrationService' `
  -ResourceGroupName $resourceGroupName `
  -location $location

$virtualNetwork = New-AzVirtualNetwork `
  -ResourceGroupName $resourceGroupName `
  -Location $location -Name $vNetName `
  -AddressPrefix $vNetAddressPrefix

Add-AzVirtualNetworkSubnetConfig `
  -Name $miSubnetName `
  -VirtualNetwork $virtualNetwork `
  -AddressPrefix $miSubnetAddressPrefix `
  -NetworkSecurityGroup $networkSecurityGroupMiManagementService `
  -RouteTable $routeTableMiManagementService |
  Set-AzVirtualNetwork

$virtualNetwork = Get-AzVirtualNetwork -Name $vNetName -ResourceGroupName $resourceGroupName
$subnet= $virtualNetwork.Subnets[0]

New-AzDataMigrationService `
  -ResourceGroupName $resourceGroupName `
  -Name $databaseMigrationService `
  -Location $location `
  -Sku Standard_2vCores `
  -VirtualSubnetId $subnet.Id

Taking a full backup with of AdventureWorks database with checksum

In [ ]:
Import-Module -Name dbatools
Backup-DbaDatabase -SqlInstance "DESKTOP-50O69FS\SQL2019" -Checksum -Type Full -AzureBaseUrl "https://cassugstorageaccount.blob.core.windows.net/cassugbacpac" -Database "AdventureWorks" -SqlCredential "https://cassugstorageaccount.blob.core.windows.net/cassugbacpac"

# Demo Starts here

## Show Assessment and Migration:

- SQL Server Management Studio
    - Deploy a Database By Using a DAC
- Azure Data Migration Assistant
    - Assessment
        - Export to Azure Migrate
    - Migration
- Azure Database Migration Service
    - Import Schema from On-premises database
    - Improt data from On-premises database
- SqlPackage generate bacpac file
    - bacpac file to Azure storage 
    - bacpac file to Local storage
    - Restore from Local storage
    - Restore from Azure storage
- Visual Studio Project (Schema only)
- Azure Data Studio Extension "Azure SQL Migration"
    - Migrate on-premises database to a Azure SQL Virtual Machine

Changing to folder where .sqlpackage binary is  
Exporting database to a .bacpac file to local storage

In [ ]:
# Delete file if exists
$FileName = "C:\temp\aw_bacpac.bacpac"

if (Test-Path $FileName) {
  Remove-Item $FileName
  write-host "$FileName has been deleted"
}

else {
  Write-host "$FileName doesn't exist"
}

In [ ]:
Set-Location -Path "C:\Program Files (x86)\Microsoft SQL Server\140\DAC\bin\"

.\SqlPackage.exe /Action:Export /SourceServerName:"DESKTOP-50O69FS\SQL2019" /SourceDatabaseName:AdventureWorks `
 /TargetFile:"C:\temp\aw_bacpac.bacpac" 

- Copying the bacpac file to storage container using [Azure Storage Explorer](https://azure.microsoft.com/en-us/features/storage-explorer/)
- You can do the same with [AzCopy](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10)
- Restoring Database using bacpac file from storage container

In [ ]:
$StorageUri = "https://cassugstorageaccount.blob.core.windows.net/cassugbacpac/aw_bacpac.bacpac"
$StorageKey = (Get-AzStorageAccountKey -ResourceGroupName $ResourceGroupName -AccountName $storageAccountName)[0].Value
$securePassword = ConvertTo-SecureString -String $password -AsPlainText -Force

$importRequest = New-AzSqlDatabaseImport `
  -ResourceGroupName $ResourceGroupName `
  -ServerName $sqlServerName  `
  -DatabaseName $databasebacpac_1 `
  -StorageKeytype $StorageKeyType `
  -StorageKey $StorageKey `
  -StorageUri $StorageUri `
  -AdministratorLogin $adminlogin `
  -AdministratorLoginPassword $securePassword `
  -Edition Standard `
  -ServiceObjectiveName "S4" `
  -DatabaseMaxSizeBytes 5368709120

In [ ]:
Get-AzSqlDatabaseImportExportStatus -OperationStatusLink $importRequest.OperationStatusLink

Restoring Database using bacpac file from local storage

Change the Azure SQL Server Name

In [ ]:
Set-Location -Path "C:\Program Files (x86)\Microsoft SQL Server\140\DAC\bin\"

.\sqlpackage.exe /a:Import /sf:C:\temp\aw_bacpac.bacpac /tsn:cassugsqlserver.database.windows.net `
/tdn:$databasebacpac_2 `
/tu:taiob@cassugsqlserver /tp:$password

Cleaning up all resources by deleting the resource group

In [ ]:
Remove-AzResourceGroup -ResourceGroupName $resourceGroupName -Force